# Wheel-enabled whole body control for Centauro
This notebook walks you through an example of how to command Centauro in a whole body fashion, 
also taking into account the ability to spin and steer its wheels. The user only needs to provide
Cartesian-space references through the Cartesio2.0 API. In this notebook, the Python ROS Client
will be used, but the analogous C++ client, or even raw topics are also an option.

## Running the robot 
If you want to send actual commands to the simulated robot, you launch the Gazebo simulator with
```
roslaunch centauro_gazebo centauro_world.launch
```
Afterwards, you usually run the *Homing* module with 
```
rosservice call /xbotcore/HomingExample_switch true
```
and
finally enable joint-space control via ros with 
```
rosservice call /xbotcore/HomingExample_switch false
rosservice call /xbotcore/XBotCommunicationPlugin_switch true
```

All the steps above can be avoided, in which case you only visualize the inverse kinematics solution in RViz.

## Running the controller
Just fire off the following
```
mon launch centauro_cartesio centauro_car_model.launch
```
Check that the controller is running by checking whether `rostopic echo /cartesian/solution` is outputting
a meaningful robot initial position.

## Visualization and interaction in Rviz
```
rviz -d `rospack find centauro_cartesio`/configs/rviz/centauro_wheeled.rviz
```
To use the interactive marker, you
 - right click on it and select *continuous mode*
 - drag around
 
## Programmatic control
We now use the CartesIO Python ROS Client library to control the robot from the code. First, we import the library and create the ROS client.

In [1]:
from cartesian_interface.pyci_all import *

cli = pyci.CartesianInterfaceRos()

By printing it, it's possible to inspect which tasks are available

In [2]:
print(cli)

CartesianInterfaceRos running inside ROS node /cartesio_ros_1587738045897279984
Defined tasks: 
 - car_frame
 - wheel_1
 - wheel_2
 - wheel_4
 - wheel_3
 - steering_wheel_1
 - steering_wheel_2
 - steering_wheel_3
 - steering_wheel_4
 - arm1_8
 - arm2_8
 - pelvis
 - rolling_wheel_1
 - rolling_wheel_2
 - rolling_wheel_3
 - rolling_wheel_4
 - ankle1_1
 - ankle1_2
 - ankle1_4
 - ankle1_3
 - Postural
 - JointLimits
 - VelocityLimits



Let us focus on **car_frame**. It provides a virtual frame of reference that is useful to control
the global position of the robot. For instance, the following code advances the robot by 1m

In [3]:
car = cli.getTask('car_frame')
cli.update()  # receive updated information from ROS
T, _, _ = car.getPoseReference()  # get current pose reference 
T.translation_ref()[0] += 1.0  # increment x coordinate by 1m
car.setPoseTarget(T, 5.0)  # command to go to T in 5 secs

True

To see that steering and rolling of the wheels is being taken into account, command a rotation

In [4]:
cli.update()  # receive updated information from ROS
T, _, _ = car.getPoseReference()  # get current pose reference
T.quaternion = [0, 0, 1, 1] # set some rotation as quaternion (will be automatically normalized)
car.setPoseTarget(T, 5.0)  # command to go to T in 5 secs

True

Other interesting tasks are
 - **pelvis** which allows for local control of the trunk of the robot
 - **arm1_8** and **arm2_8** for local control of the hands 
 
If we want to globally control a hand, we modify its base link on the fly...

In [5]:
larm = cli.getTask('arm1_8')
larm.setBaseLink('world')

True

## Further information...
... on how to use the Cartesio2.0 API can be found [here](https://advrhumanoids.github.io/CartesianInterface/)